In [32]:
from dataclasses import dataclass
import numpy as np

alphabet = "ABCDEFGH"

@dataclass
class Coordinate:
    x: int
    y: int
    z: int

class Brick():
    def __init__(self, start: Coordinate, end: Coordinate, id: int) -> None:
        self.start = start
        self.end = end
        self.id = id if id >= len(alphabet) else alphabet[id]

    def __repr__(self) -> str:
       return f"{str(self.start)} - {str(self.end)}"

In [33]:
def parse_coordinate(raw_coord):
    return Coordinate(*[int(coord) for coord in raw_coord.split(",")])

In [34]:
def pretty_print(matrix):
    print(str(matrix).replace(' ','').replace("'","").replace('[','').replace(']',''))

In [35]:
with open("day22.txt") as f:
    lines = f.readlines()
current_pattern = []
for i, line in enumerate(lines):
    if len(line) > 3:
        if i != len(lines) -1:
            line = line[:-1]
    current_pattern.append(line)
bricks = []
for brick in current_pattern:     
     start, end = brick.split("~")
     start, end = parse_coordinate(start), parse_coordinate(end)
     bricks.append(Brick(start, end, len(bricks)))
bricks


[Coordinate(x=1, y=0, z=1) - Coordinate(x=1, y=2, z=1),
 Coordinate(x=0, y=0, z=2) - Coordinate(x=2, y=0, z=2),
 Coordinate(x=0, y=2, z=3) - Coordinate(x=2, y=2, z=3),
 Coordinate(x=0, y=0, z=4) - Coordinate(x=0, y=2, z=4),
 Coordinate(x=2, y=0, z=5) - Coordinate(x=2, y=2, z=5),
 Coordinate(x=0, y=1, z=6) - Coordinate(x=2, y=1, z=6),
 Coordinate(x=1, y=1, z=8) - Coordinate(x=1, y=1, z=9)]

In [36]:
def print_dimension(bricks, dimension):
    x_dim = 3
    y_dim = 10
    symbols = np.zeros((y_dim, x_dim), dtype="str")
    for y in range(y_dim):
        for x in range(x_dim):
            symbols[y, x] = "."
    for brick in bricks:
        name = brick.id
        for x in range(getattr(brick.start, dimension), getattr(brick.end, dimension)+1):
            for z in range(brick.start.z, brick.end.z+1):
                if symbols[y_dim-1-z, x] == ".":
                    symbols[y_dim-1-z, x] = name
                else:
                     symbols[y_dim-1-z, x] = '?'
    pretty_print(symbols)

In [37]:
#print_dimension(bricks, "x")

In [38]:
#print_dimension(bricks, "y")

In [39]:
def ranges_overlap(x1, x2, y1, y2):
    return x1 <= y2 and y1 <= x2

In [40]:
while True:
    sorted_bricks = sorted(bricks, key=lambda x: x.end.z, reverse=False)
    fallen_brick = False
    for brick in sorted_bricks:
        if brick.start.z > 1:
            can_fall = True
            for other_brick in bricks:
                if brick.id != other_brick.id:
                    if other_brick.end.z == brick.start.z-1:
                        if ranges_overlap(brick.start.x, brick.end.x, other_brick.start.x, other_brick.end.x):
                            if ranges_overlap(brick.start.y, brick.end.y, other_brick.start.y, other_brick.end.y):
                                can_fall = False
                                break
            if can_fall:
                brick.start.z =  brick.start.z -1
                brick.end.z =  brick.end.z -1
                fallen_brick = True
    if not fallen_brick:
        break

In [41]:
#print_dimension(bricks, "x")

In [42]:
#print_dimension(bricks, "y")

In [43]:
supports = {}
is_supported = {}
for brick in bricks:
    supports[brick.id] = []
    is_supported[brick.id] = []
sorted_bricks = sorted(bricks, key=lambda x: x.end.z, reverse=False)
for brick in sorted_bricks:
    if brick.start.z > 1:
        for other_brick in bricks:
            if brick.id != other_brick.id:
                if other_brick.end.z == brick.start.z-1:
                    if ranges_overlap(brick.start.x, brick.end.x, other_brick.start.x, other_brick.end.x):
                        if ranges_overlap(brick.start.y, brick.end.y, other_brick.start.y, other_brick.end.y):
                            supports[other_brick.id].append(brick.id)
                            is_supported[brick.id].append(other_brick.id)

In [44]:
supports

{'A': ['B', 'C'],
 'B': ['D', 'E'],
 'C': ['D', 'E'],
 'D': ['F'],
 'E': ['F'],
 'F': ['G'],
 'G': []}

In [45]:
is_supported

{'A': [],
 'B': ['A'],
 'C': ['A'],
 'D': ['B', 'C'],
 'E': ['B', 'C'],
 'F': ['D', 'E'],
 'G': ['F']}

In [47]:
brick_to_fall = {}
for brick in bricks:
      to_remove = {brick.id}
      set_can_change = True
      current_falling = len(to_remove)
      while set_can_change:
            set_can_change = False
            for other_brick in bricks:
                  if other_brick.id not in to_remove:
                        supported = set(is_supported[other_brick.id])
                        if len(supported):
                              if len(supported - to_remove)==0:
                                    to_remove.add(other_brick.id)
            if current_falling != len(to_remove):
                  set_can_change = True
                  current_falling = len(to_remove)
      brick_to_fall[brick.id] = to_remove
sum([len(values)-1 for values in brick_to_fall.values()])
      

7